### Q1 Running Elastic Search

For elasticsearch=8.4.3:  
`version.build.hash = "42f05b9372a9a4a470db3b52817899b99a76ee73"`

### Q2 Indexing the data

#### Getting the data

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[0]

#### Indexing the data

In [ ]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework"
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(documents):
    es_client.index(
        index=index_name,
        document=doc
    )

**Q2 Answer:**    
`index` - function to add data into index

### Q3 - Searching

In [ ]:
query = 'How do execute a command on a Kubernetes pod?'

In [ ]:
search_query ={
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },

        }
    }
    }

In [ ]:
def elastic_search(search_query):
    respone = es_client.search(index=index_name, body=search_query)
    results_docs = []
    results_score = []
    for hit in respone['hits']['hits']:
        results_docs.append(hit['_source'])
        results_score.append(hit['_score'])
    return results_docs,results_score

In [ ]:
_,scores = elastic_search(search_query)
# max score: 44

**Q3 Answer:**   
`Max score = 44.5`

### Q4 Filtering

In [10]:
query = 'How do copy a file to a Docker container?'
search_query ={
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
    }

In [ ]:
search_results,_ = elastic_search(search_query)
search_results[2]

**Q4 - Answer:**  
`How do I copy files from a different folder into docker container’s working directory?`

### Q5 - Building a prompt

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    context = ""
    for doc in search_results:
        context = context + f"\nQ: {doc['question']}\nA: {doc['text']}\n\n"
    context = context.strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
query =  "How do copy a file to a Docker container?"
prompt = build_prompt(query, search_results)
len(prompt)

**Q5 Answer:**  
`1446`

### Q6 Tokens

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))

**Q6 - Answer:**  
`320`